# Flower Image Classification Parameter optimization with Keras Tuner

In this notebook, we will use the Image classification network built in the previous notebook and optimize model Hyperparameters using Keras_tuner library.


In [1]:
import keras
from skimage import io
import os
import shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
import keras_tuner as kt

In [3]:
# path to your dataset
DATASET_PATH = './flowers'
flowers_cls = ['daisy', 'roses', 'dandelion', 'sunflowers', 'tulips']

# prepare dataset for the model training

<hr>

In [4]:
for cl in flowers_cls:
  imgs_path = os.path.join(DATASET_PATH, cl) #go into dir with name of flower
  imgs = glob.glob(imgs_path + '/*.jpg') #find all pics in that dir (pic if ends in jpg)
  print(f"{cl}: {len(imgs)} Images")
  train, val = imgs[:round(len(imgs) * 0.8)], imgs[round(len(imgs) * 0.8):] #80-20 split into train/val datasets

  for t in train:
    if not os.path.exists(os.path.join(DATASET_PATH, 'train', cl)): #if a path of format flower_photos/train/[class_name] doesn't exist, make one
      os.makedirs(os.path.join(DATASET_PATH, 'train', cl)) #use makedirs to create train/[class_name] (mkdir would only create [class_name])
    shutil.move(t, os.path.join(DATASET_PATH, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(DATASET_PATH, 'val', cl)): #similar process for validation data
      os.makedirs(os.path.join(DATASET_PATH, 'val', cl)) 
    shutil.move(v, os.path.join(DATASET_PATH, 'val', cl))


daisy: 0 Images
roses: 0 Images
dandelion: 0 Images
sunflowers: 0 Images
tulips: 0 Images


In [5]:
train_dir = os.path.join(DATASET_PATH, 'train')
val_dir = os.path.join(DATASET_PATH, 'val')

In [6]:
## Define Parameters
BATCH_SIZE = 100
IMG_SHAPE = 150

In [7]:
## Data Augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 2935 images belonging to 5 classes.


In [8]:
## Validation Data Generator
val_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SHAPE, IMG_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 735 images belonging to 5 classes.


In [9]:
train_img,train_lables = train_generator.next()
val_img,val_lables = val_generator.next()

In [10]:
train_img.shape,train_lables.shape,val_img.shape,val_lables.shape

((100, 150, 150, 3), (100, 5), (100, 150, 150, 3), (100, 5))

In [11]:
## Model from previous notebook

# model = Sequential([
#     Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE, IMG_SHAPE, 3)),
#     MaxPooling2D(),
#     Conv2D(32, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
#     Conv2D(64, 3, padding='same', activation='relu'),
#     MaxPooling2D(),
#     Flatten(),
#     Dropout(0.2),
#     Dense(512, activation='relu'),
#     Dropout(0.2),
#     Dense(5, activation='softmax')
# ])

# model.compile(optimizer='adam',
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])
                

In [17]:
def model_builder(hp):
  model = keras.Sequential()
  # Convolution Layers
  model.add(keras.layers.Conv2D(filters=hp.Int("conv_1", min_value=64, max_value=128, step=32),
                                              kernel_size=hp.Choice('conv_1_kernel', values = [3,5]), 
                                              padding='same', 
                                              activation='relu', 
                                              input_shape=(IMG_SHAPE, IMG_SHAPE, 3)))
  model.add(keras.layers.Conv2D(filters=hp.Int("conv_2", min_value=32, max_value=64, step=16),
                                              kernel_size=hp.Choice('conv_2_kernel', values = [3,5]), 
                                              padding='same', 
                                              activation='relu', 
                                              input_shape=(IMG_SHAPE, IMG_SHAPE, 3)))
  # Batch Normalization Layer
  model.add(keras.layers.BatchNormalization())

  # Pooling Layer
  model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

  # Flatten Layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
  model.add(keras.layers.BatchNormalization())

  # Dropout Layer
  model.add(keras.layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

  # Output Layer
  # The parameter is set to 5 because we have 5 different classes of flowers
  # Softmax is used to convert the output to a probability distribution
  # The class with the highest probability is the models prediction
  # For example, [0.1, 0.1, 0.1, 0.6, 0.1] means the model predicts the image is a tulip
  model.add(keras.layers.Dense(5, activation='softmax'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model

In [18]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     seed=42,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [19]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.02, verbose=1)

In [20]:
tuner.search(train_img, train_lables, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



In [21]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_img, train_lables, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
3/3 [==============================] - 2s 501ms/step - loss: 5.1624 - accuracy: 0.1500 - val_loss: 7.0134 - val_accuracy: 0.1500
Epoch 2/50
3/3 [==============================] - 1s 439ms/step - loss: 1.4147 - accuracy: 0.6125 - val_loss: 3.0165 - val_accuracy: 0.1500
Epoch 3/50
3/3 [==============================] - 1s 446ms/step - loss: 0.5850 - accuracy: 0.8000 - val_loss: 2.2067 - val_accuracy: 0.2500
Epoch 4/50
3/3 [==============================] - 1s 435ms/step - loss: 0.4083 - accuracy: 0.8875 - val_loss: 2.1071 - val_accuracy: 0.2500
Epoch 5/50
3/3 [==============================] - 1s 443ms/step - loss: 0.2437 - accuracy: 0.9250 - val_loss: 1.8804 - val_accuracy: 0.2500
Epoch 6/50
3/3 [==============================] - 1s 438ms/step - loss: 0.0996 - accuracy: 0.9750 - val_loss: 1.9148 - val_accuracy: 0.2000
Epoch 7/50
3/3 [==============================] - 1s 439ms/step - loss: 0.0810 - accuracy: 0.9875 - val_loss: 1.9553 - val_accuracy: 0.2000
Epoch 8/50
3/3 [====

In [22]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_img, train_lables, epochs=best_epoch, validation_split=0.2)

Epoch 1/18
3/3 [==============================] - 2s 480ms/step - loss: 5.1094 - accuracy: 0.3625 - val_loss: 6.4467 - val_accuracy: 0.2000
Epoch 2/18
3/3 [==============================] - 1s 445ms/step - loss: 2.6814 - accuracy: 0.4625 - val_loss: 2.1317 - val_accuracy: 0.4000
Epoch 3/18
3/3 [==============================] - 1s 442ms/step - loss: 1.4762 - accuracy: 0.6750 - val_loss: 1.2480 - val_accuracy: 0.6000
Epoch 4/18
3/3 [==============================] - 1s 441ms/step - loss: 0.7003 - accuracy: 0.7500 - val_loss: 1.2817 - val_accuracy: 0.5000
Epoch 5/18
3/3 [==============================] - 1s 439ms/step - loss: 0.5878 - accuracy: 0.8000 - val_loss: 1.5092 - val_accuracy: 0.3000
Epoch 6/18
3/3 [==============================] - 1s 439ms/step - loss: 0.2601 - accuracy: 0.9250 - val_loss: 1.3550 - val_accuracy: 0.2500
Epoch 7/18
3/3 [==============================] - 1s 442ms/step - loss: 0.1442 - accuracy: 0.9625 - val_loss: 1.3304 - val_accuracy: 0.4500
Epoch 8/18
3/3 [====

In [23]:
eval_result = hypermodel.evaluate(val_img, val_lables)
print("[test loss, test accuracy]:", eval_result)

4/4 [==============================] - 0s 101ms/step - loss: 1.4840 - accuracy: 0.3400
[test loss, test accuracy]: [1.4840492010116577, 0.3400000035762787]
